# Notebook para leer los txt

Se encarga de pasar los datos de los archivos a un dataframe con la información sobre el paciente, número de repetición, el id del movimiento, si es correcto y la posición de los keypoints. Después, se crea otro dataframe con los ángulos entre algunos de los keypoits.

## 1. Raw Data Folder

* 30 participantes (14 sanos y 16 pacientes)
* 9 ejercicios
* x6 repeticiones
* 30 fps
* 2598 videos

In [9]:
# importar librerias necesarias
import pandas as pd # para manejar dataframes
import os # para interactuar con el sistema operativo
import numpy as np

### 1.1 Datos de los nombres de los archivos

SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt

* SubjectID: id uniquely identifying the person performing the exercise
* DateID: id identifying the session in which the person was recorded
* GestureLabel: Label identifying the gesture; possible values are from 0 to 8
* RepetitionNumber: Each gesture was repeated several times and this shows the repetition number
* CorrectLabel: A value of 1 represents a gesture labeled as being correctly executed, while a value of 2 is for a gesture labeled as incorrect
* Position: Some of the persons performed the gestures sitting on a chair or wheelchair, while others standing

In [10]:
# función para guardar en el dataframe los datos que aparecen en los nombre de los archivos
def leer_nombre_archivo(archivo:str):
    """
    Obtiene los datos de interés del nombre de los archivos

    :param archivo: nombre del archivo (SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt)
    :return: devuelve una lista de strings con los datos del nombre del archivo
    (SubjectID, DateID, GestureLabel, RepetitionNumber, CorrectLabel, Position)
    """ 
    archivo = archivo.split('.')[0] # quita la extension txt
    campos = archivo.split('_') # separa los campos por _
    return campos

### 1.2 Datos de los archivos
Each raw data file contains per line: timestamp, XX, XX, followed by a 25 pairs of (JointName, TrackedStatus, 3d coordinate X, 3d coordinate Y, 3d coordinate Z, 2d coordinate X, 2d coordinate Y)

In [11]:
def leer_datos_archivo(directorio:str, columnas:list[str]):

    """
    Compila toda la información de los archivos de un directorio y los guarda en un dataframe

    :param directorio: Nombre del directirio donde están los archivos
    :param columnas: Lista con el nombre de las columnas que queremos que tenga el dataframe de salida
    :return: DataFrame con todos los datos recompilados
    """
    file_list = os.listdir(directorio) # crea una lista con los nombre de los arhivos que se encuentran en ese directorio

    # Initialize an empty list to store the extracted data
    list_data = []

    # Iterate through each raw data file
    for file_name in file_list:
        campos = leer_nombre_archivo(file_name)

        with open(os.path.join(directorio, file_name), 'r') as file:
            for line in file:
                # Split the line by comma and extract the required information
                line_data = line.strip().split(',')[3:]  # Skip the initial timestamp and other unwanted data
                cleaned_data = [item.replace('(', '').replace(')', '') for item in line_data] # quitar los paréntesis
                for i in range(0, len(cleaned_data), 7): # por cada linea de los archivos necesitamos bloques de 7 valores
                    list_data.append(campos + cleaned_data[i:i + 7])
    df = pd.DataFrame(list_data, columns=columnas)
    return df


In [12]:
directory = '../dataset/SkeletonData/RawData' # directorio donde se encuentran los datos

columnas = ['SubjectID', 'DateID', 'GestureLabel', 'RepetitionNumber', 'CorrectLabel', 'Position',
            'JointName', 'TrackedStatus', '3D_X', '3D_Y', '3D_Z', '2D_X', '2D_Y']

df_data = leer_datos_archivo(directory, columnas)


In [13]:
df_data.drop(['TrackedStatus', 'DateID', '2D_X', '2D_Y'], axis=1, inplace=True)
df_data

,SubjectID,GestureLabel,RepetitionNumber,CorrectLabel,Position,JointName,3D_X,3D_Y,3D_Z
0,101,0,1,1,stand,SpineBase,-0.1028086,0.06965441,2.464606
1,101,0,1,1,stand,SpineMid,-0.1026228,0.3837799,2.438919
2,101,0,1,1,stand,Neck,-0.1025293,0.6877351,2.40196
3,101,0,1,1,stand,Head,-0.1190992,0.8358598,2.373549
4,101,0,1,1,stand,ShoulderLeft,-0.2826451,0.5525576,2.38421
...,...,...,...,...,...,...,...,...,...
5707520,307,8,9,1,stand,SpineShoulder,-0.05799517,0.5291457,2.422904
5707521,307,8,9,1,stand,HandTipLeft,-0.302538,-0.1131345,2.284269
5707522,307,8,9,1,stand,ThumbLeft,-0.2783904,-0.06298634,2.269769
5707523,307,8,9,1,stand,HandTipRight,0.1525867,-0.136378,2.45287


In [14]:
df_data.to_csv('../csvFiles/raw_pacientes.csv', index=False) # lo guardamos a cvs

## 2. Calcular los ángulos

<div>
<img src="../images/gestures.png" width="500"/>
</div>

In [15]:
def calculate_angle(df: pd.DataFrame, joint_a: str, joint_b: str, joint_c: str):
    """
    Calcula los ángulos usando tres keypoints como referencia
    """
    # Extract positions of keypoints
    positions = df.set_index('JointName')[['3D_X', '3D_Y', '3D_Z']].loc[[joint_a, joint_b, joint_c]]

    # Convert positions to numeric
    positions = positions.apply(pd.to_numeric)

    # despejar theta de u⋅v=∣u∣∣v∣cos(θ)

    # Vector u (joint_a to joint_b) and Vector v (joint_b to joint_c)
    u = np.array([positions.iloc[1, 0] - positions.iloc[0, 0], positions.iloc[1, 1] - positions.iloc[0, 1], positions.iloc[1, 2] - positions.iloc[0, 2]])
    v = np.array([positions.iloc[2, 0] - positions.iloc[1, 0], positions.iloc[2, 1] - positions.iloc[1, 1], positions.iloc[2, 2] - positions.iloc[1, 2]])
  
    # prducto vectorial y modulo de los vectores
    producto_vectorial = np.dot(u, v)
    modulo_u = np.linalg.norm(u)
    modulo_v = np.linalg.norm(v)

    # comprobar que ninguno de los modulos sea 0
    if (modulo_u * modulo_v) == 0:
        return 0

    # caclulo del coseno
    cos_angle = producto_vectorial / (modulo_u * modulo_v)

    # Calcular el angulo
    angle = np.arccos(cos_angle) * 180.0 / np.pi

    # Ensure angle is within 180 degrees
    if angle > 180.0:
        angle = 360 - angle

    return angle

In [16]:
# Group the DataFrame by every 25 rows
groups = [df_data.iloc[i:i+25] for i in range(0, len(df_data), 25)]

# Calculate angles for each group
angles = []
for group in groups:
    elbow_angle_left = calculate_angle(group, 'ShoulderLeft', 'ElbowLeft', 'WristLeft')
    elbow_angle_right = calculate_angle(group, 'ShoulderRight', 'ElbowRight', 'WristRight')
    left_arm_angle = calculate_angle(group, 'HipLeft', 'ShoulderLeft', 'ElbowLeft')
    right_arm_angle = calculate_angle(group, 'HipRight', 'ShoulderRight', 'ElbowRight')
    arms_together_angle = calculate_angle(group, 'SpineBase', 'SpineShoulder', 'WristLeft')

    elbow_angle_left = calculate_angle(group, 'ShoulderLeft', 'ElbowLeft', 'WristLeft')
    wrist_angle_left = calculate_angle(group, 'ElbowLeft', 'WristLeft', 'HandLeft')
    shoulder_angle_left = calculate_angle(group, 'ShoulderLeft', 'SpineShoulder', 'ElbowLeft')

    elbow_angle_right = calculate_angle(group, 'ShoulderRight', 'ElbowRight', 'WristRight')
    wrist_angle_right = calculate_angle(group, 'ElbowRight', 'WristRight', 'HandRight')
    shoulder_angle_right = calculate_angle(group, 'ShoulderRight', 'SpineShoulder', 'ElbowRight')

    hip_angle_left = calculate_angle(group, 'HipLeft', 'SpineBase', 'KneeLeft')
    knee_angle_left = calculate_angle(group, 'HipLeft', 'KneeLeft', 'AnkleLeft')
    ankle_angle_left = calculate_angle(group, 'KneeLeft', 'AnkleLeft', 'FootLeft')

    hip_angle_right = calculate_angle(group, 'HipRight', 'SpineBase', 'KneeRight')
    knee_angle_right = calculate_angle(group, 'HipRight', 'KneeRight', 'AnkleRight')
    ankle_angle_right = calculate_angle(group, 'KneeRight', 'AnkleRight', 'FootRight')
    
    
     # Extract additional columns
    subject_id = group['SubjectID'].iloc[0]
    gesture_label = group['GestureLabel'].iloc[0]
    repetition_number = group['RepetitionNumber'].iloc[0]
    correct_label = group['CorrectLabel'].iloc[0]
    position = group['Position'].iloc[0]
    
    # Store angles and metadata in a dictionary
    angles.append({
        'SubjectID': subject_id,
        'GestureLabel': gesture_label,
        'RepetitionNumber': repetition_number,
        'CorrectLabel': correct_label,
        'Position': position,
        'ElbowAngleLeft': elbow_angle_left,
        'ElbowAngleRight': elbow_angle_right,
        'ShoulderAngleLeft': shoulder_angle_left,
        'ShoulderAngleRight': shoulder_angle_right,
        'WristAngleLeft': wrist_angle_left,
        'WristAngleRight': wrist_angle_right,
        'HipAngleLeft': hip_angle_left,
        'KneeAngleLeft': knee_angle_left,
        'AnkleAngleLeft': ankle_angle_left,
        'HipAngleRight': hip_angle_right,
        'KneeAngleRight': knee_angle_right,
        'AnkleAngleRight': ankle_angle_right,
        'LeftArmAngle': left_arm_angle,
        'RightArmAngle': right_arm_angle,
        'ArmsTogetherAngle': arms_together_angle
    })

In [17]:
angles_df = pd.DataFrame(angles)
angles_df

,SubjectID,GestureLabel,RepetitionNumber,CorrectLabel,Position,ElbowAngleLeft,ElbowAngleRight,ShoulderAngleLeft,ShoulderAngleRight,WristAngleLeft,WristAngleRight,HipAngleLeft,KneeAngleLeft,AnkleAngleLeft,HipAngleRight,KneeAngleRight,AnkleAngleRight,LeftArmAngle,RightArmAngle,ArmsTogetherAngle
0,101,0,1,1,stand,18.731844,12.815119,142.560727,140.857143,2.598764,6.344406,104.574768,3.743551,64.246261,101.521850,4.855627,60.940986,156.791112,159.877408,152.580270
1,101,0,1,1,stand,18.682011,13.214648,142.757760,140.967721,2.522847,5.905771,104.499195,3.821802,64.334729,101.590425,4.885456,61.134582,156.667813,160.035203,152.602527
2,101,0,1,1,stand,18.530184,14.175895,142.724795,140.886094,3.239981,6.639703,104.558119,3.832570,64.078901,101.492094,4.776421,59.972082,156.747131,160.251764,152.596513
3,101,0,1,1,stand,18.525441,15.675640,142.796223,140.996819,3.488477,10.172814,104.502499,3.642046,63.451166,101.422947,4.655747,59.366231,156.715615,160.589502,152.653362
4,101,0,1,1,stand,18.440644,18.672491,142.755019,140.986030,3.640028,9.700975,104.554046,3.670732,63.212740,101.311001,4.601472,59.328043,156.793484,160.668306,152.675640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228296,307,8,9,1,stand,19.235168,16.366695,142.466122,142.949282,10.597409,8.590306,107.710008,17.152761,57.808079,94.948037,1.709497,41.616584,156.103380,158.651430,149.864559
228297,307,8,9,1,stand,19.337504,16.460773,142.614506,142.959204,9.097187,8.083043,107.352914,16.479229,57.733793,94.653231,1.263963,41.879291,156.056261,158.714378,149.734213
228298,307,8,9,1,stand,19.762179,16.292120,142.579738,142.903745,9.288112,10.157666,107.398951,16.368690,58.670153,94.346427,1.041109,78.704138,156.130164,158.836560,149.712017
228299,307,8,9,1,stand,20.339779,16.195526,142.685324,142.897308,7.335038,9.108455,107.436579,16.163032,57.798046,94.144821,1.015447,78.752737,156.114139,158.911807,149.496620


In [18]:
angles_df.to_csv('../csvFiles/angles.csv', index=False)